<a href="https://colab.research.google.com/github/handielg/Python/blob/main/NLP_Semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Clase 1**

En esta semana aprenderemos que son las incruptaciones de palabras.Para por jemplo cuando tengamos textos muy grandes, no tener que poner un código individual a cada palabra.

Podremos procesar cadenas para obtener índices de todas las palabras en un corpus de cadenas y luego convertir las cadenas en matrices de números.

Las revisiones se dividen en dos categorías principales: positivas y negativas. Así, junto con las etiquetas, TensorFlow fue capaz de construir estas incrustaciones mostrando una clara agrupación de palabras que son distintas a ambos tipos de revisión. 

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.4.0


Ahora, puede importar datasets de TensorFlow, y en este caso los llamo tfds. Con las revisiones imdb, ahora puedo llamar a tfds.load, pasarle las revisiones de cadena imdb, y devolverá los datos de imdb y los metadatos al respecto con este código. Los datos se dividen en 25.000 muestras para capacitación y 25.000 muestras para pruebas

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True) #esta biblioteca contiene  conjuntos de datos.


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHJJ8F9/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHJJ8F9/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHJJ8F9/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


**UTF-8** es el estándar Unicode,  es un sistema de codificación de caracteres diseñado para soportar el intercambio, procesamiento y visualización universal de los textos escritos en los diversos lenguajes y disciplinas técnicas del mundo moderno. Además, es compatible con los textos clásicos e históricos de muchos idiomas escritos. 

Unicode proporciona un número único a cada carácter independientemente de la plataforma, el programa o el idioma. Esto hace que bajo este estándar se agrupen más de 120.000 caracteres, todos ellos codificados para que un ordenador pueda utilizarlos en cualquier programa y sistema operativo.


In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test'] #descragamos datos de entrenamiento y test.


#Definimos las listas que contienen las frases y etiquetas, para los datos de entrenamiento y prueba.
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

#Ahora  iteramos sobre los datos de entrenamiento extrayendo las oraciones y las etiquetas. Los valores para S y l son tensores, así que al llamar a su método NumPy, realmente extraeré su valor.
# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

#Al entrenar, se espera que mis etiquetas sean matrices NumPy. Así que convertiré la lista de etiquetas que acabo de crear en matrices NumPy con este código.
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


Ahora tokenizamos nuestras oraciones.

In [ ]:
#Hiperprámetros.
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

#Importamos el tokenizador y las secuencias de pad.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) #creamos una instancia de tokenizer.pasamos nuestro tamaño de vocabulario y el token deseado fuera de vocabulario.
tokenizer.fit_on_texts(training_sentences) #encajamos nuestro tokenizador en el conjunto de datos de entrenamiento.
word_index = tokenizer.word_index #convertimos las oraciones en secuencias de números, siendo el número el valor y la palabra clave que se sacaron de las oraciones de entrenamiento.
sequences = tokenizer.texts_to_sequences(training_sentences)  #reemplazar las cadenas que contienen las palabras con el valor de token que creamos para ellas. 
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type) #rellenaremos o truncaremos las oraciones secuenciadas hasta que tengan la misma longitud, determinada por el parámetro maxlength.

testing_sequences = tokenizer.texts_to_sequences(testing_sentences) #hacemos lo mismo de la línea anterior con las secuencias de prueba.
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


Definimos nuestra red neuronal. La incrustación `(Embedding)` es la clave para el análisis e sentimientos de texto en tensorflow. La incrustación es lo siguiente: tenemos palabras en un aoración, y a menudo, palabras que tienen significados similares unas de otras. Como por ejemplo divertida y emocionante una peli. Estas palabras que se encuentran juntas reciben vectores similares. Luego con el tiempo pueden comenzar a agruparse. La red neuronal mientras aprende puede aprnder estos vectores asociándolos con las etiquetas para crear lo que se llama una incrustación, es decir, los vectores de cada palabra con su sentimiento asociado. 

Los resultados de la incrustación serán una matriz 2D con la longitud de la frase y la dimensión de la incrustación, por jemplo 16 como su tamaño. Así que las aplanamos como hicimos con las imágenes.

Luego alimentamos una red neuronal densa con esto para hacer la clasificación. 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), #esta es la línea de incrustación. 
    tf.keras.layers.Flatten(), #aplanamos 
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


Como podemos ver tenemos 10 000 palabras en nuetsro corpus y estamos trabajando en un amatriz de 16 dimensiones. 

Tal como está actualmente, nuestro índice de palabras tiene la clave siendo la palabra, y el valor es el token de la palabra. Tendremos que voltear esto, mirar a través de la lista acolchada para decodificar los tokens de nuevo en las palabras, así que hemos escrito esta función de ayuda. 

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

El proyector TensorFlow lee este tipo de archivo y lo utiliza para trazar los vectores en el espacio 3D para que podamos visualizarlos. Para el archivo vectores, simplemente escribimos el valor de cada uno de los elementos en la matriz de incrustaciones, es decir, el coeficiente de cada dimensión en el vector para esta palabra. Para la matriz de metadatos, solo escribimos las palabras. Si estás trabajando en Colab, este código descargará los dos archivos. 

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[Enlace](https://projector.tensorflow.org/)

Cargamos los archivos en este enlace para ver su representación.

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


#**Clase 2**

En este caso usamos `GlobalAveragePooling1D` en vez de `Flatten`.

In [ ]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Comenzaremos con la importación de tensorflow y json, así como las secuencias de tokenizer y pad del pre-procesamiento.

In [ ]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Ahora vamos a configurar nuestros hiperparámetros; el tamaño del vocabulario, las dimensiones de incrustación, la longitud máxima de las oraciones y otras cosas como el tamaño de entrenamiento. Este conjunto de datos tiene alrededor de 27.000 registros. Así que entrenaremos en 20.000 y validemos el resto.

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


Ahora que tiene el conjunto de datos, puede abrirlo y cargarlo como iterable con este código.

In [ ]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

Puede crear una matriz para oraciones y otra para etiquetas y, a continuación, iterar a través del almacén de datos, cargando cada título como una oración, y cada campo is_sarcástico, como su etiqueta. Dividimos entrenamiento y test.

In [ ]:
training_sentences = sentences[0:training_size] #de 0 al tamaño de entrenamiento.
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) #primero Comienza con un tokenizador, pasándole el número de palabras que desea tokenizar y el deseado fuera del token de vocabulario. 
tokenizer.fit_on_texts(training_sentences) # Luego encaja eso en el conjunto de entrenamiento llamando a ajuste en los textos, pasándolo la matriz de oraciones de entrenamiento

word_index = tokenizer.word_index #puede utilizar el texto a las secuencias para crear la secuencia de entrenamiento, reemplazando las palabras con sus tokens. 

training_sequences = tokenizer.texts_to_sequences(training_sentences) #puede utilizar el texto a las secuencias para crear la secuencia de entrenamiento, reemplazando las palabras con sus tokens. 
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type) # puede rellenar las secuencias de entrenamiento a la longitud deseada o truncar si son demasiado largas.

#haremos lo mismo, pero con un conjunto de pruebas.
testing_sequences = tokenizer.texts_to_sequences(testing_sentences) 
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type) 

Podemos crear nuestra red neuronal de la manera habitual. Lo compilaremos con entropía cruzada binaria, ya que estamos clasificando a diferentes clases.

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(), #alternativa a Flatten, que promedia a través del vector para aplanarlo.Mas y simple y un poco más rápido.
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
#entrenamos durabte 30 epochs.
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Si graficamos, podemos ver que la precisión aumenta muy bien a medida que entrenamos y la precisión de validación estaba bien, pero no muy buena. Lo interesante es que los valores de pérdida de la derecha, la pérdida de entrenamiento caen, pero la pérdida de validación aumentó.

Así que mientras que el número de predicciones precisas aumentó con el tiempo, lo interesante fue que la confianza por predicción disminuyó efectivamente. Puede que esto ocurra mucho con los datos de texto. Así que es muy importante vigilarlo. Una forma de hacerlo es explorar las diferencias a medida que modifica los hiperparámetros. Así, por ejemplo, si considera estos cambios, una disminución en el tamaño del vocabulario, y tomar oraciones más cortas, reducir la probabilidad de relleno, y luego volver a ejecutar, es posible que vea resultados como este. Aquí, se puede ver que la pérdida se ha aplanado lo que se ve bien, pero por supuesto, su precisión no es tan alta. 

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(training_padded[0]))
print(training_sentences[2])
print(labels[2])

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
sentence = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

#**Clase 3**

Vamos a echar un vistazo a una versión del conjunto de datos IMDb que ha sido pre-tokenizado para usted, pero la tokenización se realiza en palabras secundarias. Lo usaremos para demostrar cómo la clasificación de textos puede tener algunos problemas únicos, a saber, que la secuencia de palabras puede ser tan importante como su existencia.

[Enlace a datos](https://github.com/tensorflow/datasets/tree/master/docs/catalog)

In [ ]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)


In [ ]:
# Double check TF 2.0x is installed. If you ran the above block, there was a 
# 'reset all runtimes' button at the bottom that you needed to press
import tensorflow as tf
print(tf.__version__)

In [ ]:
# If the import fails, run this
# !pip install -q tensorflow-datasets

import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True) #usamos la versión 8k.


In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
tokenizer = info.features['text'].encoder

In [ ]:
#Tenemos un tokenizador de palabras secundarias preentrenadas ahora, así que podemos inspeccionar su vocabulario mirando su propiedad de sub-palabras. 

print(tokenizer.subwords)

Si queremos ver cómo codifica o decodifica cadenas, podemos hacerlo con este código. 

In [ ]:
sample_string = 'TensorFlow, from basics to mastery'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

#Entonces podemos codificar simplemente llamando al método de codificación pasándole la cadena. 
original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))


In [ ]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_data.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_data.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_data))

Definimos el modelo.

In [ ]:
embedding_dim = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
num_epochs = 10

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

Al graficar vemos que la precisión era apenas del 50 por ciento, que se podía obtener con una conjetura aleatoria. Mientras que las pérdidas disminuyen, está disminuyendo de una manera muy pequeña. Entonces, ¿por qué crees que podría ser eso? Bueno, las claves en el hecho de que estamos usando sub-palabras y no para palabras, significados de sub-palabras a menudo son absurdas y es sólo cuando los juntamos en secuencias que tienen semántica significativa. Por lo tanto, alguna manera de aprender de secuencias sería una gran manera de avanzar, y eso es exactamente lo que vas a hacer la próxima semana con redes neuronales recurrentes

#**Resumen**.

Así que aquí está el cuaderno de la lección tres. Por lo tanto, algunas cosas que debe verificar justo antes de comenzar es ir al motor de ejecución, hacer clic en cambiar el tipo de tiempo de ejecución, asegúrese de que está ejecutando Python tres y GPU. Entonces ejecuta esta celda. Si ejecuta esta celda, si no dice 2.0.0- alfa, entra en esta celda, descomente esta línea y luego ejecútelo. Pero como ya tengo 2.0.0-alfa no voy a hacer eso. Una vez que lo haya hecho, verá un botón para restablecer, haga clic en la página se restablecería, y luego puede ejecutar esta celda y simplemente asegurarse de que tiene 2.0.0-alpha antes de continuar; de lo contrario, intente volver a instalar o restablecer el tiempo de ejecución con restablecimiento de todos los tiempos de ejecución. Lo siguiente que debe hacer ahora es comenzar a cargar su conjunto de datos. Así que usamos TFDS para eso y este es el código para instalar TFDS si falla. Si lo ejecuto ahora, veremos que funcionó, no tuve ningún problema. Una vez más, si ves problemas con TFDS, entonces puedes instalarlo con este código en el tren común si vamos. Ahora puedo crear mis datos de entrenamiento y conjuntos de datos de prueba, ahora puedo crear mi tokenizer, puedo imprimir mis palabras secundarias de tokenizer, y podemos ver que hay muchos de ellos incluyendo cosas de puntuación. Para tomar una cadena de muestra como TensorFlow, desde los conceptos básicos hasta la maestría y tokenize eso, diré que mis tokens son esto. Si echo un vistazo a cómo esos tokens mapa 6307 es diez, 2327 es sor, etc Así que ahora tengo una cadena tokenizada basada en tokenización de subpalabras y construí que usando el vocabulario subwords8k.



Así que ahora vamos a crear nuestro modelo. Así que nuestro modelo solo voy a hacer otra incrustación con 64 dimensiones. Voy a pasar el tamaño del vocabulario del tokenizador en la incrustación para que podamos aprender de eso. Si entreno voy a entrenar durante 10 épocas, las épocas son bastante lentas. Voy a compilar el modelo con binario-crossentropy y agregar un optimizador, y voy a recopilar métricas de precisión, y luego mi modelo que se ajusta bastante simple. Verá que no he hecho un montón de pre-procesamiento. Es muy bueno que los datos ya estén pre-proceso para mí. Así que voy a pasar los datos de entrenamiento, voy a pasar el número de épocas, y luego voy a decir que los datos de prueba son mi prueba de datos de validación. Así que si empiezo a entrenar, veremos que es bastante lento. Así que este número será directo 25.000 registros, así que va a construir eso ya que no sabe el tamaño del récord en la primera época. Así que eso va a tardar unos momentos en construirse.
Reproduce el video desde :3: y sigue la transcripción3:00
Hemos visto que ahora se aproxima a 25.000, sabemos que hay 25.000 en la época terminará. Así que esta época tomó unos cuatro minutos.



Así que ahora podemos ver que hay más épocas. La segunda época acaba de entrar. Así que en la primera época, tuvimos unos cuatro minutos y medio, la precisión era de sólo 52 por ciento, y veremos con el tiempo que esto no va a mejorar. Voy a pausar la grabación ahora y luego volver cuando la época 10 llegue. Ahora, podemos ver la época 10 llegando a su fin, y al terminar, podemos ver que nuestros números de precisión pasaron del 52.3 por ciento al 54.2 por ciento. Bastante plano durante todo el entrenamiento. La precisión de validación es similar. Comenzó en medible y luego fue 53.18-53.49. Así que está bastante claro que nada está pasando realmente a pesar de todo el tiempo que estamos gastando en esto. Cuando trazo esto, podemos ver que mi precisión podría verse realmente bien, pero cuando miras la escala, es muy poco, y la precisión de validación es la misma. Ahora, la razón por la que esto está sucediendo, por supuesto, es sólo porque estamos trabajando en subpalabras, porque estamos entrenando en cosas que es muy difícil extraer semántica y significado de ellos y los resultados que estamos obteniendo son poco mejores que el 50 por ciento. Pero si lo piensas en un clasificador binario, una conjetura aleatoria sería del 50 por ciento. Así que esto nos lleva a un problema en el que hemos dado un pequeño paso atrás, pero está bien. A veces das un paso atrás para dar dos pasos adelante, y eso es lo que aprenderemos con RNN la próxima semana, y eso es a medida que empiezas a aprender cosas cuando se ponen en secuencia. Así que ahora sor, después de diez, diez por lo que realmente tiene algún significado, pero diez podría ser malinterpretado como el número y sor realmente no significa nada. Así que cuando empecemos a armar esto con RNs y aprender cosas donde las secuencias son importantes, entonces veremos cómo este paso atrás nos ayudará a dar un paso adelante.

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, tokenizer.vocab_size):
  word = tokenizer.decode([word_num])
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

#**Otras Notas**

##Como saber cuales son las frass más importants de un texto.

In [ ]:
!pip install rake_nltk
from rake_nltk import Rake
r = Rake()

#antes guardamos el texto en un archibo del bloc de notas, lo guardamos como todos los archivos, no como *.txt, y lo subimos.
text =  open("AAPL.txt", "r")
text = text.read()

r.extract_keywords_from_text(text)
lista_AAPL = r.get_ranked_phrases()[0:30]
lista_AAPL 

##Como saber cuantas veces se repite una palabra en un texto.

In [ ]:
#antes guardamos el texto en un archibo del bloc de notas, lo guardamos como todos los archivos, no como *.txt, y lo subimos.
texto = open("AAPL.txt", "r")
texto = texto.read()
#print(texto)


import collections

counter = collections.Counter(texto.split())
for palabra, cont in counter.most_common():
    print(f"'{palabra}' aparece {cont} {'veces' if cont > 1 else 'vez'}.")